In [20]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


In [21]:
# Load environment variables
load_dotenv()

# Initialize embeddings (same as used during indexing)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.getenv("NAGA_API_KEY"),
    base_url=os.getenv("NAGA_BASE_URL"),
)


In [22]:
# Load the database stored in the local directory
vector_store = FAISS.load_local(
    folder_path="../../Assets/Data",
    index_name="CWC_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

print(f"Vector store loaded successfully!")
print(f"Number of vectors in store: {vector_store.index.ntotal}")

# Quick test to see what's in the database
test_results = vector_store.similarity_search("cricket world cup winner", k=1)
if test_results:
    print(f"\nTest search preview:")
    print(f"First result: {test_results[0].page_content[:200]}...")
else:
    print("\n⚠️ WARNING: Vector store appears to be empty!")


Vector store loaded successfully!
Number of vectors in store: 28

Test search preview:
First result: === Results ===
The ICC released the fixture details on 27 June 2023.


== Knockout stage ==

The host India was the first team to qualify for the semi-finals after their 302-run win against Sri Lanka...


In [23]:
# Original Question
query = "Who won the 2023 Cricket World Cup?"

In [24]:
# Ranking the chunks in descending order of similarity
# Increase k to get more relevant chunks
retrieved_docs = vector_store.similarity_search(query, k=5)

# Inspect the retrieved documents
print(f"Query: {query}\n")
print(f"Number of retrieved documents: {len(retrieved_docs)}\n")
print("=" * 80)

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n📄 Document {i}:")
    print(f"Content:\n{doc.page_content}\n")
    print(f"Metadata: {doc.metadata}")
    print("=" * 80)


Query: Who won the 2023 Cricket World Cup?

Number of retrieved documents: 5


📄 Document 1:
Content:
The 2023 Cricket World Cup was hosted by India. Originally, the competition was to be played from 9 February to 26 March 2023 but due to getting longer qualification time it was moved to October to November. In July 2020, it was announced that the tournament would be moved to October and November as a result of the qualification schedule being disrupted due to the COVID-19 pandemic. The ICC released the tournament schedule on 27 June 2023. Semi-finals were played in Mumbai and Kolkata while the final was held in Ahmedabad.
India secured a place in the final for the fourth time, after defeating New Zealand in the semi-final;. having won two (1983 against the West Indies and 2011 against Sri Lanka) and lost one (2003 to current finalist Australia).

Metadata: {'title': '2023 Cricket World Cup final', 'summary': 'The 2023 Cricket World Cup Final was a One Day International cricket match p

In [25]:
# Combine multiple retrieved chunks for better context
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Debug: Print the context being used
print("Context being sent to LLM:")
print("=" * 80)
print(retrieved_context)
print("=" * 80)

# Creating the prompt
augmented_prompt = f"""Given the context below, answer the question clearly and concisely.

Context:
{retrieved_context}

Question: {query}

Answer:"""

Context being sent to LLM:
The 2023 Cricket World Cup was hosted by India. Originally, the competition was to be played from 9 February to 26 March 2023 but due to getting longer qualification time it was moved to October to November. In July 2020, it was announced that the tournament would be moved to October and November as a result of the qualification schedule being disrupted due to the COVID-19 pandemic. The ICC released the tournament schedule on 27 June 2023. Semi-finals were played in Mumbai and Kolkata while the final was held in Ahmedabad.
India secured a place in the final for the fourth time, after defeating New Zealand in the semi-final;. having won two (1983 against the West Indies and 2011 against Sri Lanka) and lost one (2003 to current finalist Australia).

The 2023 ICC Men's Cricket World Cup was the 13th edition of the ICC Men's Cricket World Cup, a quadrennial One Day International (ODI) cricket tournament organized by the International Cricket Council (ICC). It was

In [32]:
# Importing the OpenAI library from langchain
from langchain_openai import ChatOpenAI

# Instantiate the OpenAI LLM with OpenRouter (NAGA API)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("NAGA_API_KEY"),
    base_url=os.getenv("NAGA_BASE_URL")
)

# Make the API call passing the augmented prompt to the LLM
response = llm.invoke(
    [("human", augmented_prompt)]
)

In [29]:
# Extract the answer from the response object
answer = response.content

print("\n" + "=" * 80)
print("🤖 LLM RESPONSE:")
print(answer)



🤖 LLM RESPONSE:
Australia won the 2023 Cricket World Cup, defeating India in the final.
